In [1]:
!pip install googletrans

# 요약모델

# 책을 시나리오로 바꾸는 형태 Model : T5

In [ ]:
!pip install torch
!pip install transformers kobert-transformers
!pip install numpy scikit-learn


In [49]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("monologg/kobert")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)
def read_txt_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

def encode_sentences(sentences):
    embeddings = []
    for sentence in sentences:
        # 문장 토큰화 및 인코딩
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        outputs = model(**inputs)
        # [CLS] 토큰의 임베딩 사용
        embeddings.append(outputs.last_hidden_state[:, 0, :].detach().numpy())
    return np.vstack(embeddings)

def summarize_text(text, top_n=3):
    # 문장 분리 (여기선 간단히 줄바꿈으로 구분)
    sentences = [s.strip() for s in text.split('\n') if s.strip()]
    
    # 문장 임베딩 생성
    embeddings = encode_sentences(sentences)
    
    # 임베딩 간 유사도 계산
    sentence_scores = np.mean(cosine_similarity(embeddings), axis=1)
    
    # 상위 점수의 문장 선택
    top_indices = np.argsort(sentence_scores)[-top_n:]
    top_sentences = [sentences[i] for i in sorted(top_indices)]
    
    # 시나리오 형식으로 반환
    return "\n".join(f"Scene {i+1}: {s}" for i, s in enumerate(top_sentences))

# 텍스트 파일 경로
file_path = "C:/Users/User/Desktop/나노마신.txt"

# 텍스트 읽기
original_text = read_txt_file(file_path)

# 요약하기
scenario_summary = summarize_text(original_text, top_n=10)

# 요약 결과 출력
print("시나리오 형태 요약:")
print(scenario_summary)


시나리오 형태 요약:
Scene 1: 멈춰선 소년은 후들거리는 다리를 붙들고 짜증스러운 표정으로 그들을 노려보았다.
Scene 2: 복면인들의 말에 천 공자라 불린 소년은 어이가 없다는 표정을 지었다.
Scene 3: 살기를 머금은 그들의 눈빛은 소년의 목과 심장을 향하고 있었다.
Scene 4: 천한 피라는 말에 소년의 눈에 살의가 치솟았다.
Scene 5: 하지만 이미 소년의 단검을 쥐고 있는 자세만 보더라도 어설프기 짝이 없었다.
Scene 6: 일그러진 얼굴에 그의 단검을 떨어뜨리게 만든 복면인의 눈이 히죽거리더니, 재빠른 손놀림으로 소년의 목을 움켜쥐었다.
Scene 7: 완전히 제압했다고 방심했던 복면인의 턱으로 단검이 파고들어왔다.
Scene 8: 그가 잘못 본 것이 아니라면 웬 하얀 빛줄기가 날아와서 복면인을 맞췄는데, 상반신이 녹아내리듯이 사라져버린 것이었다.
Scene 9: 복면인 대주가 남은 복면인에게 퇴각 신호를 보내려는 순간 그의 상반신이 흰 빛줄기에 맞고 사라져버렸다.
Scene 10: 죽어가는 소년의 몸에 변화가 일어났다.
